In [57]:
import pickle as pkl
def save_data(fname, data):
    output = open(fname, 'wb')
    pkl.dump(data, output)
    output.close()  
    
def load_data(fname):
    inp = open(fname, 'rb')
    d = pkl.load(inp)
    inp.close()
    return d

import nltk.corpus
from nltk import decorators
import nltk.stem
import string 
import pymorphy2

punctuations = list(string.punctuation)
#stemmer = nltk.stem.snowball.RussianStemmer()
stopwords = nltk.corpus.stopwords.words('russian')
stopwords += punctuations
stopwords += [u'--', u'...', u'—', u'г.', u'тасс', u'риа', u'', u'из-за', u'новость']
morph = pymorphy2.MorphAnalyzer()

# remove all tokenz that:
# 1. occur only in one doc
# 2. Occur more than in 30%(x%) of docs

def clear_token(token):
    return token.replace(u"«", u'').replace(u"»", u'').replace(u"`", u'').replace(u"'", u'').replace(u"/", u'')

def is_ok_token(token):
    if token in stopwords: return False
    if not any(c.isalpha() for c in token): return False
    return True
    

def stem_tokens(tokens):
    return [clear_token(x) for x in [morph.parse(item)[0].normal_form for item in tokens] if is_ok_token(x)]


'''remove punctuation, lowercase, stem'''
def normalize(text):
    t = text.lower()
    return stem_tokens(nltk.word_tokenize(t))

def print_topics(topics):
    idx = 1
    for l in topics:
        print "{}>>> ".format(idx),
        idx += 1
        for t in l:
            print u"{}:{} ".format(t[0], t[1]),
        print
        print
        
def topic_msgs(topic, num=10):
    cnt = 0
    for j in topics[topic]:
        cnt += 1
        print u"{}({}):{}".format(j[0], j[1], res[j[0]])
        if cnt > num: break 

import os
import codecs
from tqdm import tqdm, tqdm_notebook

def walkdir(folder):
    """Walk through each files in a directory"""
    for dirpath, dirs, files in os.walk(folder):
        for filename in files:
            yield os.path.abspath(os.path.join(dirpath, filename))

def get_texts_from_dir(dname, extension=None):
    texts = []
    fls = 0
    read = 0
    proceed = 0
    wrong_enc = 0

    # calculate total number
    filecounter = 0
    for filepath in walkdir(dname):
        filecounter += 1
    # set bar object
    t = tqdm_notebook(walkdir(dname), total=filecounter, unit="files")
    for fname in t:
        fls += 1
        if extension is not None:
            if not fname.endswith(extension):
                continue
        read += 1
        try:
            fl = codecs.open(fname, encoding='utf-8')
            body = fl.read()
        except UnicodeDecodeError:
            wrong_enc += 1
            continue
        if body.startswith(u"Document Outline"):
            continue
        proceed += 1
        texts.append((fname, normalize(body)))
        t.set_postfix(Files=fls, Read=read, Proceed=proceed, Wrong_encoded=wrong_enc)
    return texts

In [58]:
# create texts collection
texts = get_texts_from_dir('/Volumes/Backup/Calibre/Sept-2018/', '.txt')
save_data('/Volumes/Backup/Calibre/texts.pkl', texts)

5446/|/  8%|| 5446/66280 [5:02:44<56:21:39,  3.34s/files, Files=5.45e+03, Proceed=494, Read=1.07e+03, Wrong_encoded=5]

KeyboardInterrupt: 

In [ ]:
# make dict and corpus
from gensim import corpora, models, similarities

dictionary = corpora.Dictionary(texts)
dictionary.filter_extremes(no_below=2, no_above=0.3, keep_n=None)
dictionary.compactify()
corpus = [dictionary.doc2bow(text) for text in texts]

# lda
lda = models.ldamodel.LdaModel(corpus, id2word=dictionary, num_topics=100, alpha='auto',  passes=5, iterations=10)

# viz
import pyLDAvis.gensim
data = pyLDAvis.gensim.prepare(lda, corpus, dictionary)
pyLDAvis.display(data)

In [29]:
"Document Outline".startswith('qwe')

False

In [53]:
filecounter = 0
for filepath in walkdir('/Volumes/Backup/Calibre/Sept-2018/'):
    filecounter += 1

In [56]:
t = tqdm_notebook(walkdir('/Volumes/Backup/Calibre/Sept-2018/'), total=66280, unit="files")
for fname in t:
    t.set_postfix(File=fname)
    sleep(1)

NameError: name 'sleep' is not defined

In [59]:
len (texts)

0

In [ ]:
1